# 🧠 RAG (Retrieval-Augmented Generation) 튜토리얼

이 노트북에서는 RAG가 무엇인지, 왜 필요한지 실습을 통해 알아봅니다.

## 1️⃣ 문제: LLM은 모든 것을 알지 못한다

먼저 LLM에게 우리 회사에 대해 물어봅시다.

In [8]:
import httpx
import os
from dotenv import load_dotenv

load_dotenv()
LLM_BASE_URL = os.getenv("LLM_BASE_URL", "http://localhost:11434")
LLM_MODEL = os.getenv("LLM_MODEL", "qwen2.5vl:72b")

# LLM에 직접 질문
def ask_llm(question):
    response = httpx.post(
        f"{LLM_BASE_URL}/api/generate",
        json={"model": LLM_MODEL, "prompt": question, "stream": False},
        timeout=60.0
    )
    return response.json()["response"]

# 우리 회사에 대해 질문
question = "PFCT가 뭐야?"
answer = ask_llm(question)
print(f"질문: {question}")
print(f"답변: {answer}")

질문: PFCT가 뭐야?
답변: "PFCT"는 "Perfect"의 약자로, 완벽한 것을 의미합니다. 이 용어는 주로 인터넷에서 사용되며, 특히 게임이나 스포츠에서 완벽한 성과를 달성했을 때 사용됩니다. 예를 들어, 어떤 게임에서 모든 레벨을 완벽하게 클리어했을 때 "PFCT"라고 말할 수 있습니다.


**결과**: LLM은 학습 데이터에 없는 최신 정보나 회사 내부 정보를 모릅니다.

## 2️⃣ 해결책: RAG - 문서를 제공하면 정확하게 답변

이제 PFCT에 대한 정보를 문서로 제공하고 다시 질문해봅시다.

In [9]:
# PFCT 회사 정보 문서
document = """
PFCT(PFC Technologies, 피에프씨테크놀로지스)는 대한민국의 선도적인 핀테크(FinTech) 기업입니다.

주요 사업:
- 크플(cple): 온라인투자연계금융(온투금융) 플랫폼으로 대출자와 투자자를 연결
- 에어팩(AIRPACK): AI 기반 신용 리스크 관리 B2B 솔루션

서비스 목표:
- 금융 취약 계층에게 합리적인 중금리 대출 기회 제공
- 투자자에게 연 10% 내외의 안정적인 투자 수익 제공

주요 성과:
- 대한민국 온투금융사 1호 등록
- 2022년 Forbes Asia 100 to Watch 선정
- 비은행권 최고의 AI 신용평가 기술력 보유

연혁:
- 2024년 3월: 피플펀드컴퍼니에서 피에프씨테크놀로지스로 사명 변경
- 플랫폼 브랜드명도 '피플펀드'에서 '크플'로 통합
"""

# 문서와 함께 질문
def ask_with_context(question, context):
    prompt = f"""다음 문서를 참고하여 질문에 답변하세요.

문서:
{context}

질문: {question}

답변:"""
    return ask_llm(prompt)

answer = ask_with_context("PFCT가 뭐야?", document)
print(f"답변: {answer}")

답변: PFCT는 PFC Technologies의 약자로, 한국의 선도적인 핀테크(FinTech) 기업입니다. 이 회사는 온라인투자연계금융(온투금융) 플랫폼인 '크플(cple)'과 AI 기반 신용 리스크 관리 B2B 솔루션인 '에어팩(AIRPACK)'을 주요 사업으로 하고 있습니다. PFCT의 목표는 금융 취약 계층에게 합리적인 중금리 대출 기회를 제공하고, 투자자에게 안정적인 투자 수익을 제공하는 것입니다. 이 회사는 대한민국 온투금융사 1호 등록, 2022년 Forbes Asia 100 to Watch 선정, 비은행권 최고의 AI 신용평가 기술력 보유 등의 주요 성과를 이루었습니다. 2024년 3월에 피플펀드컴퍼니에서 피에프씨테크놀로지스로 사명을 변경하고, 플랫폼 브랜드명도 '피플펀드'에서 '크플'로 통합하였습니다.


**결과**: 문서를 제공하니 정확한 답변을 합니다! 이것이 RAG의 핵심입니다.

## 3️⃣ RAG 시스템 사용하기

이제 우리의 RAG 시스템을 사용해봅시다.

In [10]:
from ontology_rag.core.rag_engine import RAGEngine
from ontology_rag.embeddings.client import EmbeddingClient
from ontology_rag.storage.vector_store import VectorStore

# RAG 엔진 초기화
embedder = EmbeddingClient(base_url=LLM_BASE_URL)
store = VectorStore()
rag = RAGEngine(
    llm_base_url=LLM_BASE_URL,
    llm_model=LLM_MODEL,
    embedding_client=embedder,
    vector_store=store,
)

print("✅ RAG 엔진 초기화 완료")

✅ RAG 엔진 초기화 완료


In [11]:
# 회사 정보 문서 추가
documents = [
    """PFCT(PFC Technologies, 피에프씨테크놀로지스)는 대한민국의 선도적인 핀테크 기업입니다.
    온라인투자연계금융 플랫폼 '크플(cple)'을 운영하며, 대출자와 투자자를 연결합니다.""",
    
    """PFCT의 주요 제품:
    1. 크플(cple): 온투금융 플랫폼, 중금리 대출과 투자 연결
    2. 에어팩(AIRPACK): AI 기반 신용 리스크 관리 B2B 솔루션
    3. 비은행권 최고의 AI 신용평가 기술력 보유""",
    
    """PFCT 주요 성과:
    - 대한민국 온투금융사 1호 등록
    - 2022년 Forbes Asia 100 to Watch 선정
    - 2024년 3월 피플펀드컴퍼니에서 피에프씨테크놀로지스로 사명 변경
    - 투자자에게 연 10% 내외의 안정적인 수익 제공"""
]

rag.add_documents(documents)
print(f"✅ {len(documents)}개 문서 추가 완료")

✅ 3개 문서 추가 완료


In [12]:
# RAG로 질문하기
question = "PFCT가 뭐야?"
answer = rag.query(question, top_k=2, debug=True)
print(f"\n질문: {question}")
print(f"답변: {answer}")

Search results: {'ids': [['doc_0', 'doc_1']], 'embeddings': None, 'documents': [["PFCT(PFC Technologies, 피에프씨테크놀로지스)는 대한민국의 선도적인 핀테크 기업입니다.\n    온라인투자연계금융 플랫폼 '크플(cple)'을 운영하며, 대출자와 투자자를 연결합니다.", 'PFCT의 주요 제품:\n    1. 크플(cple): 온투금융 플랫폼, 중금리 대출과 투자 연결\n    2. 에어팩(AIRPACK): AI 기반 신용 리스크 관리 B2B 솔루션\n    3. 비은행권 최고의 AI 신용평가 기술력 보유']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[None, None]], 'distances': [[96.59842681884766, 151.52749633789062]]}
Found 2 documents
Doc 1: PFCT(PFC Technologies, 피에프씨테크놀로지스)는 대한민국의 선도적인 핀테크 기업입니다.
    온라인투자연계금융 플랫폼 '크플(cple)'을 운영하며, 대출자와 투
Doc 2: PFCT의 주요 제품:
    1. 크플(cple): 온투금융 플랫폼, 중금리 대출과 투자 연결
    2. 에어팩(AIRPACK): AI 기반 신용 리스크 관리 B2B 솔루션
 

질문: PFCT가 뭐야?
답변: PFCT는 대한민국의 선도적인 핀테크 기업입니다. 이 회사는 온라인투자연계금융 플랫폼 '크플(cple)'을 운영하며, 대출자와 투자자를 연결합니다. 또한, PFCT는 AI 기반 신용 리스크 관리 B2B 솔루션 '에어팩(AIRPACK)'을 제공하며, 비은행권 최고의 AI 신용평가 기술력을 보유하고 있습니다.


## 4️⃣ RAG가 작동하는 방식

1. **문서 저장**: 문서를 벡터(숫자 배열)로 변환하여 저장
2. **질문 검색**: 질문과 유사한 문서를 찾음
3. **답변 생성**: 찾은 문서를 LLM에 제공하여 답변 생성

이렇게 하면 LLM이 모르는 정보도 정확하게 답변할 수 있습니다!

In [13]:
# 다양한 질문 시도
questions = [
    "크플이 뭐야?",
    "에어팩은 무슨 제품이야?",
    "PFCT의 주요 성과는?",
    "피플펀드와 PFCT의 관계는?"
]

for q in questions:
    answer = rag.query(q, top_k=2)
    print(f"\n질문: {q}")
    print(f"답변: {answer}")
    print("-" * 80)


질문: 크플이 뭐야?
답변: 크플(cple)은 PFCT가 운영하는 온라인투자연계금융 플랫폼으로, 대출자와 투자자를 연결합니다. 또한, 이 플랫폼은 중금리 대출과 투자를 연결하는 역할을 합니다.
--------------------------------------------------------------------------------

질문: 에어팩은 무슨 제품이야?
답변: 에어팩(AIRPACK)은 AI 기반 신용 리스크 관리 B2B 솔루션입니다.
--------------------------------------------------------------------------------

질문: PFCT의 주요 성과는?
답변: 문서에서 PFCT의 주요 성과에 대한 정보를 찾을 수 없습니다. 문서에는 PFCT의 주요 제품과 서비스에 대한 설명이 있지만, 구체적인 성과나 성과 지표에 대한 내용은 포함되어 있지 않습니다.
--------------------------------------------------------------------------------

질문: 피플펀드와 PFCT의 관계는?
답변: 문서에서 해당 정보를 찾을 수 없습니다.
--------------------------------------------------------------------------------


## 5️⃣ 문서가 없으면?

RAG는 관련 문서가 없으면 "정보를 찾을 수 없다"고 답변합니다.

In [14]:
# 문서에 없는 내용 질문
question = "양자컴퓨터가 뭐야?"
answer = rag.query(question, top_k=2)
print(f"질문: {question}")
print(f"답변: {answer}")

질문: 양자컴퓨터가 뭐야?
답변: 문서에서 해당 정보를 찾을 수 없습니다.


## 🎯 정리

- **문제**: LLM은 학습하지 않은 정보(회사 내부 정보, 최신 정보)를 모름
- **해결**: RAG로 관련 문서를 제공하여 정확한 답변 생성
- **장점**: 
  - 최신 정보 반영 가능
  - 회사 내부 지식 활용
  - 환각(hallucination) 방지
  - 출처 추적 가능

## 💡 활용 사례

- 사내 문서 검색 및 Q&A
- 고객 지원 챗봇
- 규정/매뉴얼 기반 답변
- 금융 상품 설명

이제 여러분의 문서를 추가하고 RAG를 활용해보세요! 🚀